In [222]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from polygon.rest import RESTClient
import json
import statsmodels.api as sm
from datetime import datetime
import pytz
from tqdm import tqdm
import concurrent.futures
from scipy.optimize import minimize
from cvxopt import solvers, matrix

api_key = 'hFrBS7nzcaLTa8mplO1ejm44DI4EscDM'
client = RESTClient(api_key)

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
warnings.simplefilter("ignore", category=UserWarning)

<h1>Load Data</h1>

In [223]:
df = pd.read_csv('fullsample.csv')
df

,Unnamed: 0,ticker,date,pred,trade
0,27219,BVXV,2021-01-04,-0.441402,0
1,151092,VINC,2021-01-04,0.233248,0
2,3737,AEZS,2021-01-04,-0.878808,0
3,136483,SSTK,2021-01-04,-0.253395,0
4,110749,PED,2021-01-04,1.059537,0
...,...,...,...,...,...
156407,3659,AEVA,2024-09-27,0.967824,0
156408,147874,UONE,2024-09-27,1.835434,1
156409,5223,AIFF,2024-09-27,1.853510,1
156410,139626,TANH,2024-09-27,0.167393,0


In [224]:
def est(unix_ms_timestamp):
    # Convert Unix timestamp in milliseconds to seconds
    unix_seconds = unix_ms_timestamp / 1000.0
    # Create a datetime object from the Unix timestamp
    utc_time = datetime.utcfromtimestamp(unix_seconds)
    # Define the UTC and EST timezones
    utc_zone = pytz.utc
    est_zone = pytz.timezone('US/Eastern')
    # Localize the UTC datetime object to UTC timezone
    utc_time = utc_zone.localize(utc_time)
    # Convert the UTC time to EST
    est_time = utc_time.astimezone(est_zone)
    est_time = est_time.replace(tzinfo=None)
    return est_time.strftime('%Y-%m-%d')

In [225]:
class single_factor_model():
    def __init__(self,index_name: str, riskfree_filename: str) -> None:
        '''Load index/factor, trading dates, and riskfree data'''
        self.index_data = self.load_index(index_name)    #will replace factor variable access with different method once using multiple factors
        with open('dlist.json', 'r') as file:
            self.dlist = json.load(file)
        rf_df = pd.read_csv(riskfree_filename)
        rf_df['riskfree'] = rf_df['DGS3MO']/365
        rf_df.rename(columns={'observation_date': 'date'}, inplace=True)
        self.riskfree = rf_df[['date','riskfree']]

    def load_index(self, index_name: str) -> pd.DataFrame:
        '''Collect index data, compute overnight percentage change'''
        data = client.get_aggs(index_name,multiplier=1,timespan='day',adjusted='true',from_='2015-01-01',to='2024-11-30',limit = 50000)
        df = pd.DataFrame(data)
        df['index_overnight'] = ((df['open'].shift(-1)/df['close'])-1) * 100
        df['date'] = df['timestamp'].apply(est)
        return df[['date','index_overnight']]
    
    def ticker_data(self, ticker, end_date):
        '''Collect and clean single ticker data, compute ticker and index risk-adjusted returns, match by date'''
        start_date = self.dlist[self.dlist.index(end_date) - 252]
        data = client.get_aggs(ticker,multiplier=1,timespan='day',adjusted='true',from_=start_date,to=end_date,limit = 50000)
        df = pd.DataFrame(data)
        df['date'] = df['timestamp'].apply(est)
        #fill dates not present (didn't trade on day, leads to incorrect overnight returns if not addressed)
        date_range = [date for date in self.dlist if start_date <= date <= end_date]
        df = pd.merge(pd.DataFrame({'date': date_range}), df, on='date', how='left')
        df['ticker_overnight'] = ((df['open'].shift(-1)/df['close'])-1) * 100   #overnight returns
        #add index data, excess returns
        df = pd.merge(df,self.index_data,on='date')
        df = pd.merge(df,self.riskfree,on='date')
        df['ticker_return'] = df['ticker_overnight'] - df['riskfree']   #excess returns
        df['index_return'] = df['index_overnight'] - df['riskfree']
        return df
    
    def factor_model(self,ticker: str, end_date: str, proj_return: float) -> tuple[float,float,float]:
        '''Regress ticker returns on index returns and compute alpha, beta, and idiosyncratic variance'''
        df = self.ticker_data(ticker,end_date)
        df = df[['ticker_return','index_return']].dropna()
        model = sm.OLS(df['ticker_return'],sm.add_constant(df['index_return'])).fit()
        alpha = model.params[0] + proj_return
        beta = model.params[1]
        i_var = model.resid.var()
        return alpha,beta,i_var
    
    def parallel_collection(self, dataframe: pd.DataFrame)-> pd.DataFrame:
        '''Collect ticker statistics(alpha, beta, variance) from factor model with parrallel processing
            Takes input dataframe with ticker,date,projected return columns (see names 2 lines below)
            Returns dataframe with alpha,beta,variance for each ticker'''
        with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
            factor_outcomes = list(tqdm(executor.map(self.factor_model, dataframe['ticker'], dataframe['date'], dataframe['pred']), total=len(dataframe)))
        factor_dataframe = pd.DataFrame(factor_outcomes, columns=['alpha', 'beta', 'i_var'])
        return factor_dataframe
    
    def index_variance(self, end_date: str)-> float:
        '''Return index variance (will become covariance matrix with multiple factors)'''
        return self.ticker_data('AAPL',end_date)['index_return'].var()
    
    def compile_results(self,dataframe: pd.DataFrame)-> tuple[np.array, np.array]:
        '''Collect alpha,beta,idiosyncratic variance for all tickers and factor variance
            Return alpha for all tickers, sigma covariance matrix'''
        df = self.parallel_collection(dataframe)
        df.loc[len(df)] = [0,1,0]   #add index stats
        #create sigma cov matrix
        i = np.diag(df['i_var'])
        index_var = self.index_variance(dataframe['date'].iloc[0])
        sigma = (np.outer(df['beta'], df['beta']) * index_var) + i
        return np.array(df['alpha']),sigma

In [226]:
class quadprog():
    def __init__(self,dataframe: pd.DataFrame, lambda2: int)-> pd.DataFrame:
        '''Takes input dataframe, must include columns: ticker, date, projected return (date columns must all contain same value)
            Collects alpha and sigma input matrices, maximizes u(w) = w^T(alpha) - lambda2 * (w^T * sigma * w)'''
        #Collect factor model data
        self.l2 = lambda2
        fm = single_factor_model('SPY','DGS3MO.csv')
        self.alpha, self.sigma = fm.compile_results(dataframe)
        self.tickers = dataframe['ticker'].tolist() + ['SPY']

    def optimize(self)-> None:
        ''' Determine optimal weights by maximizing: u(w) = w^T(alpha) - lambda2 * (w^T * sigma * w)
            -> Subject to: sum(abs(wi)) = 1, wi <= .2, cannot short anything but index(SPY)
            (1)Equivalent to minimize u(w) = lambda2 * (w^T * sigma * w) - w^T(alpha)
            (2)In order to compute absolute value constraints, seperate w into positive and negative components (w_pos,w_neg)
                (2a)Operation becomes: minimize lambda2 * ((w_pos - w_neg)^T * sigma * (w_pos - w_neg)) - (w_pos - w_neg)^T(alpha)
            (3)Setup for (.5 * x^T * P * x) + (x^T * q)
                (3a)Decision x = [w_pos,w_neg] -> 2n by 1
                (3b)Quadratic P = [[(2 * l2 * sigma),(-2 * l2 * sigma)], [(-2 * l2 * sigma),(2 * l2 * sigma)]] -> 2n by 2n, obtained from (2a) expansion
                (3c)Linear q = [-alpha,alpha] -> 2n by 1, obtained from (2a) expansion
            (4)Equality constraint: absolute value of weights must sum to 1. sum(w_posi + w_negi) = 1, encoded as A * x = b
                (4a)A_sum = [1,1,...,1] -> 1 by 2n
                (4b)b_sum = [1] -> 1 by 1
            (5)Inequality constraints
                (5a)Max position size 20% of portfolio. w_posi + w_negi <= .2, encoded as G_abs * x = h_abs
                    G_abs: diagnol coefficient matrix with 1s-> n by 2n
                    h_abs = [.2,.2,...,.2] -> n by 1
                (5b)w_pos and w_neg must be non-negative. w_posi >= 0, w_negi >= 0, encoded as G_nn * x = h_nn
                    G_nn: diagnol coefficient matrix with -1s-> 2n by 2n
                    h_nn = [0,0,0] -> 2n by 1
                (5c)Only instrument that can be shorted is index(SPY). w_negi - w_posi <= 0 for all i not representing index(SPY), encoded as G_restict * x <= h_restrict
                    G_restrict: diagnol coefficient matrix with -1s for w_pos, 1s for w_neg -> (n-1) by 2n
                    h_restrict = [0,0,...,0] -> (n-1) by 1
                (5d)Combine all G and h terms to enforce all inequality constraints'''
        n = len(self.alpha)
        #3b: quadratic term P
        P = np.zeros((2 * n, 2 * n))
        P[:n, :n] = 2 * self.l2 * self.sigma 
        P[n:, n:] = 2 * self.l2 * self.sigma
        P[:n, n:] = -2 * self.l2 * self.sigma
        P[n:, :n] = -2 * self.l2 * self.sigma
        P = matrix(P)
        #3c: linear term q
        q = np.zeros(2 * n)
        q[:n] = -self.alpha  
        q[n:] = self.alpha   
        q = matrix(q)
        #4: equality constraint, absolute value of weights must sum to 1
        A = np.ones((1, 2 * n))
        b = np.array([1.0])
        A = matrix(A)
        b = matrix(b)
        #5a: inequality constraint, max position size = 20% portfolio
        G_abs = np.zeros((n, 2 * n))
        for i in range(n):
            G_abs[i, i] = 1
            G_abs[i, n + i] = 1
        h_abs = np.full(n, 0.2)
        #5b: inequality constraint, w_pos and w_neg non-negative
        G_nonneg = np.zeros((2 * n, 2 * n))
        np.fill_diagonal(G_nonneg, -1)
        h_nonneg = np.zeros(2 * n)
        #5c: inequality constraint, can only short SPY, 
        G_restrict_neg = np.zeros((n - 1, 2 * n))
        for i in range(n - 1):
            G_restrict_neg[i, i] = -1  # w^+_i >= 0
            G_restrict_neg[i, n + i] = 1  # w^-_i <= 0
        h_restrict_neg = np.zeros(n - 1)
        #5d: combine inequality constraints
        G = np.vstack([G_abs, G_nonneg, G_restrict_neg])
        h = np.hstack([h_abs, h_nonneg, h_restrict_neg])
        G = matrix(G)
        h = matrix(h)
        #Solve
        solvers.options['show_progress'] = False
        solvers.options['abstol'] = 1e-9
        solvers.options['reltol'] = 1e-9
        solvers.options['feastol'] = 1e-9
        solution = solvers.qp(P, q, G, h, A, b)
        w_plus_opt = np.array(solution['x'][:n]).flatten()
        w_minus_opt = np.array(solution['x'][n:]).flatten()
        self.decimal_weights = w_plus_opt - w_minus_opt
            
    def scale_weights(self,portfolio_value: int)-> pd.DataFrame:
        '''Convert decimal weights into dollar values based on portfolio size'''
        dollar_weights = portfolio_value * self.decimal_weights
        return pd.DataFrame({'ticker': self.tickers, 'dollar_weight': [round(dw, 6) for dw in dollar_weights]})

In [232]:
qp = quadprog(df[df['date'] == '2024-09-23'],0)
qp.optimize()
a = qp.scale_weights(10000)
a.sort_values(by='dollar_weight',ascending=False)

100%|██████████| 169/169 [00:04<00:00, 39.03it/s]


,ticker,dollar_weight
125,EONR,2000.000000
24,STSS,2000.000000
99,GXAI,1999.999999
154,NUKK,1999.999997
40,MYND,1999.999994
...,...,...
59,SKYE,0.000000
60,HOOK,0.000000
61,TMC,0.000000
62,SONM,0.000000
